In [1]:
pip install pyodbc

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pyodbc
import pandas as pd

# Conexão
server = "ekp-svr-datafinance.database.windows.net"
database = "BD_Dados_Finance"
username = "admin-db"
password = "Pedrica1009@"
driver = '{ODBC Driver 18 for SQL Server}'  # certifique-se que esse driver está instalado

#Conectar
conn = pyodbc.connect(
    f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;'
)

cursor = conn.cursor()

# Exemplo: criar tabela
cursor.execute("""
IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='NVDA' AND xtype='U')
BEGIN
    CREATE TABLE NVDA (
        Id INT PRIMARY KEY IDENTITY(1,1),
        ticket NVARCHAR(10),
        year INT NOT NULL,
        month INT NOT NULL,
        day INT NOT NULL,
        time TIME,
        open FLOAT NOT NULL,
        high FLOAT NOT NULL,
        low FLOAT NOT NULL,
        close FLOAT NOT NULL,
        volume FLOAT NOT NULL
    )
END
""")
conn.commit()

dados = pd.read_csv("03 - Gold\\dados_load.csv")

# 5. Inserir no banco
for _, row in dados.iterrows():
    cursor.execute("""
        INSERT INTO NVDA (ticket, year, month, day, time, open, high, low, close, volume)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)""", 
        (
    "NVDA",           # ticket
    row['year'],      # year
    row['month'],     # month
    row['day'],       # day
    row['time'],      # time
    row['open'],      # open
    row['high'],      # high
    row['low'],       # low
    row['close'],     # close
    row['volume']     # volume
))

# Exemplo: ler dados
cursor.execute("SELECT TOP 10 * FROM NVDA")
for row in cursor.fetchall():
    print(row)

# Fechar conexão
cursor.close()
conn.close()


(1, 'Enzo', 22)
